In [36]:
B=1400
GLM_name='Poisson'  # or your model
n = 700  # Main sample size
p = 10   # Number of features
r = int(n**0.9)  # Size of each sub-sample
maintain=1
folder='resultspart1'

In [37]:
import pandas as pd
import  numpy as np
import torch
# def f_1(x):

#     val = (
#         - x[:, 0]
#         + x[:, 1]
#         + x[:, 2] * x[:, 3]
#         + 2.0 / (1.0 + abs(x[:, 4]))
#     )
    

#     if isinstance(val, torch.Tensor):
#         val = val.clone()
#     else:
#         val = val.copy()
    

#     val[val > 2]  =  2
#     val[val < -2] = -2
    
#     return val
def sigmoid(x):
    return 1/(1+np.exp(-x))

# def f_1(x):
#     return - x[:, 0] + x[:, 1] +x[:,2]*x[:,3]+2/(1+abs(x[:,4]))

# def f_1(x):
#     return x[:, 0] + 0.25 * x[:, 1] ** 2 

def f_1(x):
    return x[:, 0] + 0.25 * x[:, 1] ** 2+0.1*torch.tanh(0.5*x[:,2]-0.3)

def f_2(x):
    return x[:,0]+abs(x[:,1]*x[:,2])-torch.exp(0.2*x[:,3])+torch.log(1+abs(x[:,4])*2)


# 1) Read CSVs (no header)
df_bf1 = pd.read_csv(f"{folder}/{GLM_name}fBf1Kerneln{n}p{p}B{B}r{r}.csv", header=None)
df_sd  = pd.read_csv(f"{folder}/{GLM_name}sdf1Kernel{n}p{p}B{B}r{r}.csv", header=None)
df_sdcrt  = pd.read_csv(f"{folder}/{GLM_name}sdcrtf1Kernel{n}p{p}B{B}r{r}.csv", header=None)
xtest = torch.load(f"{folder}/xtest{p}.pt")
# 2) Convert to NumPy arrays, then to torch.Tensor
#    (you can specify dtype=torch.float32 if needed)
tensor_bf1 = torch.tensor(df_bf1.values, dtype=torch.float32)
for col in range(tensor_bf1.shape[1]):
    column = tensor_bf1[:, col]
    
    # 找出有限值（排除 inf, -inf, nan）
    finite_mask = torch.isfinite(column)
    
    if finite_mask.any():  # 如果这一列有有限值
        max_val = column[finite_mask].max()
        
        # 替换 inf、-inf 和 nan 为最大值
        invalid_mask = ~finite_mask  # 也就是 inf 或 nan
        column[invalid_mask] = max_val
tensor_sd  = torch.tensor(df_sd.values,  dtype=torch.float32)
tensor_sdcrt  = torch.tensor(df_sdcrt.values,  dtype=torch.float32)

truef = torch.tensor(f_1(xtest), dtype=torch.float32)

if GLM_name=="Bernoulli":
    def activate(x):
        return 1/(1+np.exp(-x))
    
    true_f=truef.numpy()

    estimated_f = tensor_bf1.numpy()
    estimated_f0=estimated_f.mean(axis=0)
    quantile=np.quantile(np.abs(estimated_f0-true_f),maintain)
    index=(np.abs(estimated_f0-true_f)<=quantile)
    estimated_f = estimated_f[:,index]
    estimated_f0=estimated_f.mean(axis=0)
    std=tensor_sd.numpy()[:,index]
    stdcrt=tensor_sdcrt.numpy()[:,index]
    min_non_nan = np.nanmin(stdcrt)          # 忽略 NaN 求最小
    stdcrt = np.nan_to_num(stdcrt, nan=min_non_nan)
    min_non_nan = np.nanmin(std)   
    std = np.nan_to_num(std, nan=min_non_nan)
    std0=np.std(estimated_f,axis=0)
    std1=np.mean(stdcrt,axis=0)
    true_f=true_f[index]
   
    cover=((true_f >= estimated_f-1.96*stdcrt) & (true_f <= estimated_f+1.96*stdcrt))
    cover0=((true_f >= estimated_f-1.96*std0) & (true_f <= estimated_f+1.96*std0))
    cover1=((true_f >= estimated_f-1.96*std1) & (true_f <= estimated_f+1.96*std1))
    AIL=np.mean(activate(estimated_f+1.96*stdcrt)-activate(estimated_f-1.96*stdcrt))
    AILsd=np.std(activate(estimated_f+1.96*stdcrt)-activate(estimated_f-1.96*stdcrt))
    
    # within_confidence_interval = ((true_f >= estimated_f-1.96*std) & (true_f <= estimated_f+1.96*std))
elif GLM_name=="Poisson":
    def activate(x):
        return np.exp(x)
    true_f=np.log(np.log(1+np.exp(truef.numpy())))
    estimated_f = tensor_bf1.numpy()
    estimated_f0=estimated_f.mean(axis=0)
    quantile=np.quantile(np.abs(estimated_f0-true_f),maintain)
    index=(np.abs(estimated_f0-true_f)<=quantile)

    estimated_f = estimated_f[:,index]
    estimated_f0=estimated_f.mean(axis=0)
    std=tensor_sd.numpy()[:,index]
    stdcrt=tensor_sdcrt.numpy()[:,index]
    stdcrt = np.nan_to_num(stdcrt, nan=0.0)
    std = np.nan_to_num(std, nan=0.0)
    std0=np.std(estimated_f,axis=0)
    std1=np.mean(std,axis=0)
    true_f=true_f[index]
    cover=((true_f >= estimated_f-1.96*stdcrt) & (true_f <= estimated_f+1.96*stdcrt))
    AIL=np.mean(activate(estimated_f+1.96*stdcrt)-activate(estimated_f-1.96*stdcrt))
    AILsd=np.std(activate(estimated_f+1.96*stdcrt)-activate(estimated_f-1.96*stdcrt))
    
    # cover0=((true_f >= estimated_f-1.96*std0) & (true_f <= estimated_f+1.96*std0))
    # cover1=((true_f >= estimated_f-1.96*std1) & (true_f <= estimated_f+1.96*std1))
    # true_lambda=np.exp(true_f)
    # estimated_lambda=np.exp(estimated_f)
    
    # within_confidence_interval = ((true_f >= estimated_f-1.96*std) & (true_f <= estimated_f+1.96*std))

    # # Step 2: Count the number of times true_probability is within the confidence interval
    # count_within_interval = within_confidence_interval.sum()
else: 
    def activate(x):
        return 1/(1+np.exp(-x))
    true_f=truef.numpy()

    estimated_f = tensor_bf1.numpy()
    estimated_f0=estimated_f.mean(axis=0)
    quantile=np.quantile(np.abs(estimated_f0-true_f),maintain)
    index=(np.abs(estimated_f0-true_f)<=quantile)
    estimated_f = estimated_f[:,index]
    estimated_f0=estimated_f.mean(axis=0)
    std=tensor_sd.numpy()[:,index]
    stdcrt=tensor_sdcrt.numpy()[:,index]
    std0=np.std(estimated_f,axis=0)
    stdcrt = np.nan_to_num(stdcrt, nan=0.0)
    std = np.nan_to_num(std, nan=0.0)
    std1=np.mean(stdcrt,axis=0)
    true_f=true_f[index]
   
    cover=((true_f >= estimated_f-1.96*stdcrt) & (true_f <= estimated_f+1.96*stdcrt))
    cover0=((true_f >= estimated_f-1.96*std0) & (true_f <= estimated_f+1.96*std0))
    cover1=((true_f >= estimated_f-1.96*std1) & (true_f <= estimated_f+1.96*std1))
    AIL=np.mean(activate(estimated_f+1.96*stdcrt)-activate(estimated_f-1.96*stdcrt))
    AILsd=np.std(activate(estimated_f+1.96*stdcrt)-activate(estimated_f-1.96*stdcrt))


print("biasf:",np.mean(estimated_f0-true_f),"(sd):",np.std(estimated_f-true_f) )
print("MAEf:",np.mean(np.abs(estimated_f-true_f)) ,"(sd):", np.std(np.abs(estimated_f-true_f)) )
print("biaspsi:",np.mean((activate(estimated_f)-activate(true_f))),"sd:",np.std((activate(estimated_f)-activate(true_f))) )
print("MAEpsi:",np.mean(np.abs(activate(estimated_f)-activate(true_f))),"SD:",np.std(np.abs(activate(estimated_f)-activate(true_f))) )
print('The empirical std:',np.mean(std0))
print('SE:',np.mean(std))
print('SE_c:',np.mean(stdcrt))
print("coverprob",np.mean(np.mean(cover,axis=0)))
print("AIL:",AIL)
print("AILsd:",AILsd)



biasf: -0.101824835 (sd): 0.30485952
MAEf: 0.25631708 (sd): 0.1939309
biaspsi: -0.05296065 sd: 0.23859051
MAEpsi: 0.18137465 SD: 0.16380933
The empirical std: 0.2852274
SE: 0.3649118
SE_c: 0.29180318
coverprob 0.9190833333333334
AIL: 0.8735855
AILsd: 0.5037429


/tmp/ipykernel_1414045/3113603942.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  truef = torch.tensor(f_1(xtest), dtype=torch.float32)
